In [53]:
import numpy as np
import pandas as pd
from scipy.linalg import eigh

In [54]:
# X is a matrix of size (n, d)
# n is the number of samples
# d is the number of features
n = 100
d = 10
X = np.random.randn(n, d)

epsilon = 1e-6

In [55]:
mu = X.mean()
X = X - mu

def f1(X):
    N = X.shape[0]
    _, Sigma, Vt = np.linalg.svd(X, full_matrices=False)
    s = (Sigma ** 2) / N
    # compute transpose of Vt
    return s[::-1], Vt.transpose()[:, ::-1]
    # return s, Vt.transpose()

def f2(X):
    C = (1 / X.shape[0]) * np.dot(X.transpose(), X)
    s, E = eigh(C)
    return s, E

def sign_ambiguity(E):
    return pd.DataFrame(E * np.sign(np.diag(E)))

s1, E1 = f1(X)
s2, E2 = f2(X)

E1 = sign_ambiguity(E1)
E2 = sign_ambiguity(E2)


In [56]:
print("Are eigenvalues almost equal?", np.allclose(s1, s2))

print("Are eigenvectors almost equal?", np.allclose(E1.values.flatten(), E2.values.flatten()))

Are eigenvalues almost equal? True
Are eigenvectors almost equal? True


In [115]:

import numpy as np
n = 5 
d = 9
X = np.random.randn(n, d)
mu = X.mean(axis=0)
X = X - mu
epsilon = 1e-6
N = X.shape[0]
_, Sigma, Vt = np.linalg.svd(X, full_matrices=True)

# compute the rank of the matrix X
r = np.linalg.matrix_rank(X)

assert (r == n - 1) | (r == d), "X is not full rank"

# if n <= d then Sigma has shape (n,) so it will need to be expanded to do and filled with the 
# values of the last element of Sigma
if n <= d:
    assert Sigma.shape[0] == n
    assert (r == n - 1)
    Sigma = np.concatenate((Sigma[0:r], np.repeat(Sigma[r-1], d-r)))

W = (Vt / Sigma[:, np.newaxis]).transpose() * np.sqrt(N-1)
Xw = X @ W

# Compute the covariance matrix of Xw
C = np.dot(Xw.transpose(), Xw) / (N-1)

if r == d:
    print("Is C the identity matrix?", np.allclose(C, np.eye(d)))
    print("Is C a diagonal matrix?", np.allclose(C, np.diag(np.diag(C))))
else:
    print("Is the first r rows and columns of C the identity matrix?", np.allclose(C[0:r, 0:r], np.eye(r)))
    print("Is the last d-r rows and columns of C the zero matrix?", np.allclose(C[r:, r:], np.zeros((d-r, d-r))))



Is the first r rows and columns of C the identity matrix? True
Is the last d-r rows and columns of C the zero matrix? True


In [55]:
A = Vt.transpose() @ np.diag(1 / Sigma)
B = (Vt / Sigma[:, np.newaxis]).transpose()
print("Are A and B almost equal?", np.allclose(A, B))
# But method B is more efficient than method A so that's what we do in practice

Are A and B almost equal? True


In [58]:
import numpy as np
import pandas as pd
# X is a matrix of size (n, d)
# n is the number of samples
# d is the number of features
n = 100
d = 10
X = np.random.randn(n, d)
Y = np.random.randn(n, d)
# select the last 20 samples as the reference population
Xc = X[-20:, :]
Yc = Y[-20:, :]

epsilon = 1e-6

Zc = np.concatenate((Xc, Yc), axis=0)
# cast as a dataframe
Zc = pd.DataFrame(Zc)

from pycytominer.operations.transform import Spherize

scalerZ = Spherize(method="PCA", center=True)
scalerZ = scalerZ.fit(Zc)
Zcs1 = scalerZ.transform(Zc)

# verify that Zs1 has identity covariance matrix
print("Is the covariance matrix of Zs1 the identity matrix?", np.allclose(Zcs1.cov(), np.eye(d)))

# Now get back the original Xc and Yc
# Use the size of Xc and Yc to slice Zs to get back Xs and Ys
Xcs1 = Zcs1.iloc[:Xc.shape[0], :]
Ycs1 = Zcs1.iloc[Xc.shape[0]:, :]

# now compute scalers for Xs and Ys
scalerX = Spherize(method="PCA", center=True)
scalerX = scalerX.fit(Xcs1)
scalerY = Spherize(method="PCA", center=True)
scalerY = scalerY.fit(Ycs1)

# now transform X and Y
Xcs2 = scalerX.transform(Xcs1)
Ycs2 = scalerY.transform(Ycs1)

# verify that Xs2 and Ys2 have identity covariance matrices
print("Is the covariance matrix of Xs2 the identity matrix?", np.allclose(Xcs2.cov(), np.eye(d)))
print("Is the covariance matrix of Ys2 the identity matrix?", np.allclose(Ycs2.cov(), np.eye(d)))

# Now transform all rows of X and Y
Xs1 = scalerZ.transform(pd.DataFrame(X))
Xs2 = scalerX.transform(Xs1)

Ys1 = scalerZ.transform(pd.DataFrame(Y))
Ys2 = scalerY.transform(Ys1)


Is the covariance matrix of Zs1 the identity matrix? True
Is the covariance matrix of Xs2 the identity matrix? True
Is the covariance matrix of Ys2 the identity matrix? True
